In [ ]:
import pandas as pd

: 

In [ ]:
!which python

: 

In [5]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

ModuleNotFoundError: No module named 'kaggle'

In [ ]:
# Download the dataset
api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions', 'RAW_recipes.csv', unzip=True)
data = pd.read_csv('RAW_recipes.csv')
